In [1]:
import os

os.environ['HTTP_PROXY'] = "http://proxy-internet-aws-eu.subsidia.org:3128"
os.environ['HTTPS_PROXY'] = "http://proxy-internet-aws-eu.subsidia.org:3128"
os.environ['no_proxy'] = "169.254.169.254,127.0.0.1"

In [2]:
%%sh

sudo su

cat <<EOF >> /etc/sysconfig/docker
export HTTPS_PROXY=http://proxy-internet-aws-eu.subsidia.org:3128
export HTTP_PROXY=http://proxy-internet-aws-eu.subsidia.org:3128
export NO_PROXY=169.254.169.254,127.0.0.1
EOF

service docker restart

Stopping docker: [  OK  ]
Starting docker:	.[  OK  ]


## Build Docker images for train & serve

### Build generic train image : MXNET + Gluonts
*Do not execute cell, image already exists in ECR*

In [ ]:
%%sh

export HTTPS_PROXY=http://proxy-internet-aws-eu.subsidia.org:3128
export HTTP_PROXY=http://proxy-internet-aws-eu.subsidia.org:3128
export NO_PROXY=169.254.169.254,127.0.0.1

# Image name
image_name=mxnet-gluonts-training-0.3.3-cpu

cd container
account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-eu-west-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${image_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${image_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --registry-ids ${account} --no-include-email) #for docker you must authenticate both accounts, dkt and amazon
$(aws ecr get-login --region ${region} --registry-ids 763104351884 --no-include-email) #for amazon

# Build the docker image locally with the image name and then push it to ECR
# with the full name.
docker build  --no-cache --pull --file 'Dockerfile_gluonts_training_0.3.3_cpu' -t ${image_name} .
docker tag ${image_name} ${fullname}
docker push ${fullname}

### Build training image ( from gluonts image + src code )

In [25]:
%%sh

export HTTPS_PROXY=http://proxy-internet-aws-eu.subsidia.org:3128
export HTTP_PROXY=http://proxy-internet-aws-eu.subsidia.org:3128
export NO_PROXY=169.254.169.254,127.0.0.1

# The name of our algorithm
algorithm_name=apo-demand-forecast-training

cd container
chmod +x train
#chmod -R +x src/

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-eu-west-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --registry-ids ${account} --no-include-email) #for docker you must authenticate both accounts, dkt and amazon

# Build the docker image locally with the image name and then push it to ECR
# with the full name.
docker build  --no-cache --pull --file 'Dockerfile_train' -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}
docker push ${fullname}

Login Succeeded

Step 1/4 : FROM 150258775384.dkr.ecr.eu-west-1.amazonaws.com/mxnet-gluonts-training-0.3.3-cpu:latest
latest: Pulling from mxnet-gluonts-training-0.3.3-cpu
Digest: sha256:060009925c389f20c3be2b291cf1032097c7aaedff213456c789ed19a315056a
Status: Image is up to date for 150258775384.dkr.ecr.eu-west-1.amazonaws.com/mxnet-gluonts-training-0.3.3-cpu:latest
 ---> 31ee196156f7
Step 2/4 : COPY src/ /opt/program
 ---> 904a637293dc
Step 3/4 : COPY config/ /opt/program
 ---> 45f0fda84718
Step 4/4 : ENTRYPOINT ["python"]
 ---> Running in cafff8eb9172
Removing intermediate container cafff8eb9172
 ---> 16186096397d
Successfully built 16186096397d
Successfully tagged apo-demand-forecast-training:latest
The push refers to repository [150258775384.dkr.ecr.eu-west-1.amazonaws.com/apo-demand-forecast-training]
59da816ce36f: Preparing
fb19b57e9e1e: Preparing
2b3fd2c0e44f: Preparing
98db11483f2b: Preparing
3190daa7d5b8: Preparing
8dfd6a9b4a23: Preparing
76f3ba8c633a: Preparing
6994e2dcf65c: 

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [26]:
from sagemaker import get_execution_role

In [27]:
role = get_execution_role()
role

'arn:aws:iam::150258775384:role/FORECAST-SAGEMAKER-DEV'

In [28]:
bucket = 'fcst-workspace'
project_id = 'qlik/data/refined/domyos_nov_2019/gluonts_data' #/train_data_cutoff_201922.csv

## Bring your own Docker Image Sagemaker

In [29]:
image_name = '150258775384.dkr.ecr.eu-west-1.amazonaws.com/apo-demand-forecast-training:latest'

In [30]:
hyperparameters = {
        'yearly_order': 26,
        'quaterly_order': 5,
    
        'weekly_seasonality': False,
        'daily_seasonality': False,
        'yearly_seasonality': False,
    
        'n_changepoints': 36,
        'changepoint_range': 0.69,
        'changepoint_prior_scale': 1.91,
        'seasonality_prior_scale': 2.04
}

In [31]:
from sagemaker.estimator import Estimator
#from sagemaker.estimator import EstimatorBase

In [32]:
instance_type = 'ml.m5.xlarge' #'local''ml.m5.2xlarge' #'local'#'ml.p3.2xlarge' # #'local' #'local_gpu'

estimator = Estimator(role=role,
                      train_instance_count=1,
                      train_instance_type=instance_type,
                      image_name=image_name,
                      hyperparameters=hyperparameters,
                      security_group_ids=["sg-0186b5ab868f43e42"],
                      subnets=["subnet-0f87a7ed73f4ead6d", "subnet-02c60aed04f0d4ee5"] #, entry_point='src/train.py'
                      )

estimator.fit('s3://'+bucket+'/'+project_id) #'/gluonts_ds_cutoff_201922.pkl' #job_name

2020-01-16 10:12:36 Starting - Starting the training job...
2020-01-16 10:12:38 Starting - Launching requested ML instances...
2020-01-16 10:13:34 Starting - Preparing the instances for training......
2020-01-16 10:14:30 Downloading - Downloading input data...
2020-01-16 10:14:41 Training - Downloading the training image......
2020-01-16 10:15:49 Training - Training image download completed. Training in progress.downloaded fcst-workspace/qlik/data/clean/active_sales.csv to: /opt/ml/input/data/active_sales.csv
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
Initial log joint probability = -5.13485
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       532.347      0.055997       29.5402           1           1      117   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       542.498       0.01723        64.864      0.7143      0.7143      222   
    I

Initial log joint probability = -5.32497
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       658.283   0.000725405       285.471      0.5936      0.5936      118   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       666.339    0.00308687       114.721           1           1      228   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       673.835    0.00868013       65.9807           1           1      334   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       675.827     0.0511714       53.1112           1           1      437   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       677.159    0.00148991       21.5797           1           1      543   
    Iter      log prob        ||dx||      ||grad||       alpha  

Initial log joint probability = -5.845
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       255.716      0.056639       16.1716           1           1      117   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       256.301    0.00114808       5.73669      0.7753      0.7753      234   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       256.555    0.00635002       9.55986      0.9902      0.9902      348   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       256.733    0.00184129       3.50607      0.1408           1      462   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       257.044   0.000306776        4.8381      0.6246      0.6246      583   
    Iter      log prob        ||dx||      ||grad||       alpha    

    2899       526.029    0.00090798       49.8155           1           1     3203   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
    2999       526.418    0.00137086       9.63114           1           1     3316   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
    3099       526.661   4.12761e-05       9.17826      0.6403      0.6403     3432   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
    3199       526.766    0.00305436       15.5778           1           1     3544   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
    3299       527.087   0.000725242       13.8423           1           1     3659   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
    3399       527.362   0.000309482       34.5305     0.06057           1     3769   
    Iter      log 

Initial log joint probability = -9.84085
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99        477.41    0.00484736       93.3887           1           1      116   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       490.814     0.0947728       163.217           1           1      225   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       497.302     0.0727316       90.2618       1.526      0.1526      339   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       499.109    0.00195197       14.8864           1           1      449   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499        502.86    0.00536437       39.2062           1           1      559   
    Iter      log prob        ||dx||      ||grad||       alpha  

...

    8399       474.333   5.56119e-06       1.74336      0.4764           1     9752   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
    8408       474.333    9.6906e-07       1.76105      0.8673      0.8673     9764   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance
Initial log joint probability = -3.19074
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       422.438     0.0227573       335.212       0.887       0.887      115   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       434.384     0.0107626       146.721      0.4981           1      221   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299        443.95       0.00331       44.3627           1           1      333   
    Iter      log prob        ||dx||      ||grad||

Initial log joint probability = -3.55708
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       421.364     0.0273661       86.3227       1.705      0.1705      123   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       429.778    0.00480928       99.7117           1           1      231   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       438.982     0.0129093       27.2065           1           1      344   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       443.233    0.00605184       116.052           1           1      449   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       445.376     0.0377725        59.423           1           1      558   
    Iter      log prob        ||dx||      ||grad||       alpha  

    5599       479.634    0.00494885       64.5538           1           1     6189   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
    5699       479.982    0.00321978       26.9572           1           1     6303   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
    5799       480.273    0.00138436       19.8852       0.789       0.789     6409   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
    5899       480.524   0.000576076       24.9846           1           1     6518   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
    5999       480.667    0.00014536       9.95875           1           1     6627   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
    6099        480.86   0.000852147       33.3128      0.7293      0.7293     6743   
    Iter      log 


2020-01-16 10:18:51 Uploading - Uploading generated training model
2020-01-16 10:18:51 Failed - Training job failed


UnexpectedStatusException: Error for Training job apo-demand-forecast-training-2020-01-16-10-12-35-555: Failed. Reason: AlgorithmError: Exit Code: 1

## Bring your own hyperopt

In [11]:
import boto3

from time import gmtime, strftime
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

In [20]:
tuning_job_name = 'demand-forecast-prophet-tuning'# + strftime("%d-%H-%M-%S", gmtime())


hyperparameter_ranges = {
        'yearly_order': IntegerParameter (26, 29),#, #(1, 30)
        'quaterly_order': IntegerParameter(4, 6)#, #(1, 10)
}    
#        'n_changepoints': IntegerParameter(30, 32), #(1, 50)
#        'changepoint_range': ContinuousParameter(0.65, 0.69), #(0.6, 1.)
#        'changepoint_prior_scale': ContinuousParameter(1.8, 1.9, scaling_type="Logarithmic"), #1e-2, 1e2
#        'seasonality_prior_scale': ContinuousParameter(2.2, 2.4, scaling_type="Logarithmic")
#}


In [21]:
objective_metric_name = 'global_wape'
objective_type = 'Minimize'
metric_definitions = [{'Name': 'global_wape',
                       'Regex': 'global_wape: ([0-9\\.]+)'}]

In [22]:
tuner = HyperparameterTuner(estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=9,
                            max_parallel_jobs=1,
                            objective_type=objective_type)

In [23]:
#tuner.fit({'training': 's3://'+bucket+'/'+project_id,
#          'test': 's3://'+bucket+'/'+project_id})

tuner.fit('s3://'+bucket+'/'+project_id)

In [24]:
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

'InProgress'